In [2]:
import json
import re
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os
import pandas as pd
import datetime
import subprocess
import csv
import time

# set the client_ID, client_SECRET, redirect_uri and username for the spotify api authentication
client_ID = '4e1c1626b9e04c0fba6d8f14d31ab3e6'
client_SECRET = '607dd5362f9d4f44b33361eca5aa81b8'   
redirect_uri = 'http://127.0.0.1:9090'
username = 'rt47etgc6xpwhhhb8575rth83'

# data_folder = r"C:\Users\saket\Documents\1.MY_DATA\spotify\spotify api data"
recently_played_file_name = 'recently_played_tracks.csv'

client_credentials_manager = spotipy.oauth2.SpotifyOAuth(
        # scope=scope,
        username=username,
        client_id=client_ID,
        client_secret=client_SECRET,
        redirect_uri=redirect_uri,
        # open_browser=False
        )

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


# returns a spotipy object with the given scope
def get_spotify_token(scope):
    client_credentials_manager.get_access_token(as_dict=False)
    return spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# token to modify user's playlists
playlist_modify_public = get_spotify_token("playlist-modify-public")

# user-read-recently-played
user_read_recently_played = get_spotify_token("user-read-recently-played")

# user-top-read
user_top_read = get_spotify_token("user-top-read")

# playlist-modify-private
playlist_modify_private = get_spotify_token("playlist-modify-private")

# user-library-read
user_library_read = get_spotify_token("user-library-read")

# playlist-read-private
playlist_read_private = get_spotify_token("playlist-read-private")

# get all the tracks from a private playlist
def get_playlist_tracks(playlist_id):
    results = playlist_read_private.user_playlist_tracks(username, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

# returns the recently played tracks
def get_recently_played_tracks(limit=50):
    recently_played = user_read_recently_played.current_user_recently_played(limit=limit)
    return recently_played

# returns the top 50 artists played by the user
def add_to_playlist(playlist_id, list_of_tracks):
    splity_split = split_with_no(list_of_tracks, 100)
    for hundred_lis in splity_split:
        playlist_modify_private.user_playlist_add_tracks(user=username, playlist_id=playlist_id, tracks=hundred_lis)

# splits the list into sublists of size 100
def split_with_no(list, No):
    if type(list) == str:
        return [list]
    lists = []
    no = 0
    listy = []
    for i in list:
        if no < 100:
            no += 1
            listy.append(i)
        else:
            lists.append(listy) 
            listy = []
            no = 0
    lists.append(listy)
    print('len - ',len(lists))
    return lists

# returns the top 50 tracks played by the user
def get_top_tracks(limit=50):
    top_tracks = token.current_user_top_tracks(limit=limit)
    return top_tracks

# returns a datetime object from the given timestamp
def timestamp_to_time(timestamp):
    try:
        # Try to parse the timestamp string as ISO format with microsecond component
        dt_object = datetime.datetime.fromisoformat(timestamp[:-1])
    except ValueError:
        try:
            # Try to parse the timestamp string as ISO format without microsecond component
            dt_object = datetime.datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%SZ')
        except ValueError:
            # Try to parse the timestamp string as a Unix timestamp in seconds
            dt_object = datetime.datetime.fromtimestamp(int(timestamp))

    return dt_object

# returns a dictionary with the track info for the given track
def get_track_info_from_json(track_dict):
    # iterate through all the keys in the track_json try to get the value of the key, else set the value to None
    track_info = {}
    for key in track_dict.keys():
        try:
            track_info[key] = track_json[key]
        except:
            track_info[key] = None
    return track_info

# returns a list of dictionaries with the track info for each track
def get_recently_played_tracks_info():
    # list to store the recently played tracks
    recently_played_tracks = []
    # get the recently played tracks
    recently_played = get_recently_played_tracks()
    # iterate through all the recently played tracks
    for i in range(len(recently_played['items'])):
        # get the track info
        track = recently_played['items'][i]['track']
        # create a dictionary to store the track info
        track_info = {
            'track_name': track['name'],
            'track_id': track['id'],
            'track_duration': track['duration_ms'],
            'track_popularity': track['popularity'],
            'track_explicit': track['explicit'],
            'album_name': track['album']['name'],
            'album_id': track['album']['id'],
            'album_release_date': track['album']['release_date'],
            'album_release_date_precision': track['album']['release_date_precision'],
            'album_total_tracks': track['album']['total_tracks'],
            'album_type': track['album']['album_type'],
            'artist_name': track['artists'][0]['name'],
            'artist_id': track['artists'][0]['id'],
            'played_at': recently_played['items'][i]['played_at']
        }
        # print the track info
        song_name = track['name']
        # print the track time and name
        # print(recently_played['items'][i]['played_at'], song_name)

        recently_played_tracks.append(track_info)
    return recently_played_tracks

# stores the recently played tracks in a csv file
def store_recently_played_tracks():
    # get the recently played tracks
    recently_played_tracks = get_recently_played_tracks_info()
    # create a dataframe from the stored recently played tracks
    old_recently_played_tracks = pd.read_csv(recently_played_file_name)
    # create a dataframe from the list of recently played tracks
    new_recently_played_tracks = pd.DataFrame(recently_played_tracks)
    # get the last played track from the old_recently_played_tracks
    last_played_at = old_recently_played_tracks['played_at'].iloc[-1]
    # reverse the new_recently_played_tracks
    new_recently_played_tracks = new_recently_played_tracks.iloc[::-1]

    # iterate though all the new_recently_played_tracks and find the one which is closest to the last_played_at
    for i in range(len(new_recently_played_tracks)):
        # get the difference between the last_played_at and the current track's played_at in hours
        diff = (timestamp_to_time(last_played_at) - timestamp_to_time(new_recently_played_tracks['played_at'].iloc[i])).total_seconds() / 3600
        # if the difference is negative, then the current track is the one which was last played
        if diff < 0:
            # get the index of the new_recently_played_tracks
            index = i
            # start the new_recently_played_tracks from the index
            new_recently_played_tracks = new_recently_played_tracks.iloc[index:]
            # iterate through all the new_recently_played_tracks and print the track name and time
            print("Newly added tracks:")
            for j in range(len(new_recently_played_tracks)):
                print(new_recently_played_tracks['played_at'].iloc[j], new_recently_played_tracks['track_name'].iloc[j])
            # add the new_recently_played_tracks to the old_recently_played_tracks, use .concat() to avoid the index being repeated
            old_recently_played_tracks = pd.concat([old_recently_played_tracks, new_recently_played_tracks])
            # save the old_recently_played_tracks to the csv file
            old_recently_played_tracks.to_csv(recently_played_file_name, index=False)
            break

# function to get the spotify token for the given scope
def get_top_tracks(limit=200):
    top_tracks = user_top_read.current_user_top_tracks(limit=limit, time_range='short_term')
    return top_tracks

# returns the top tracks info in a list
def get_top_tracks_info():
    top_tracks = get_top_tracks()
    top_tracks_info = []
    for i in range(len(top_tracks['items'])):
        track = top_tracks['items'][i]
        track_info = {
            'track_name': track['name'],
            'track_id': track['id'],
            'track_duration': track['duration_ms'],
            'track_popularity': track['popularity'],
            'track_explicit': track['explicit'],
            'album_name': track['album']['name'],
            'album_id': track['album']['id'],
            'album_release_date': track['album']['release_date'],
            'album_release_date_precision': track['album']['release_date_precision'],
            'album_total_tracks': track['album']['total_tracks'],
            'album_type': track['album']['album_type'],
            'artist_name': track['artists'][0]['name'],
            'artist_id': track['artists'][0]['id']
        }
        top_tracks_info.append(track_info)
    return top_tracks_info

# returns the user's top artists in a list
def get_top_artists_info(limit=200):
    top_artists = user_top_read.current_user_top_artists(limit=limit)
    top_artists_info = []
    for i in range(len(top_artists['items'])):
        artist = top_artists['items'][i]
        artist_info = {
            'artist_name': artist['name'],
            'artist_id': artist['id'],
            'artist_popularity': artist['popularity'],
            'artist_followers': artist['followers']['total'],
            'artist_genres': artist['genres']
        }
        top_artists_info.append(artist_info)
    return top_artists_info

# returns the user's top genres in a list
def get_top_genres_info(limit=200):
    top_artists = user_top_read.current_user_top_artists(limit=limit)
    top_genres_info = []
    for i in range(len(top_artists['items'])):
        artist = top_artists['items'][i]
        for j in range(len(artist['genres'])):
            genre = artist['genres'][j]
            genre_info = {
                'genre': genre
            }
            top_genres_info.append(genre_info)
    return top_genres_info

# returns the user's library tracks in a list
def get_library_info(limit=200):
    # get the token for the user-library-read scope
    # get the user's library data
    library = user_library_read.current_user_saved_tracks(limit=limit)

    # convert the library data to a dataframe
    library_df = pd.DataFrame(library)
    return library_df

# returns a list of dictionaries containing the user's playlists
def get_user_playlists():
    user_playlists = playlist_read_private.current_user_playlists(limit=50)
    return user_playlists

# returns a list of dictionaries containing the user's playlists
def store_user_playlists_info():
    user_playlists_info = get_user_playlists_info()
    user_playlists_info_df = pd.DataFrame(user_playlists_info)
    user_playlists_info_df.to_csv('user_playlists_info.csv', index=False)

# returns the tracks in a playlist in a dictionary
def get_user_playlist_tracks(playlist_id):
    user_playlist_tracks = playlist_read_private.user_playlist_tracks(playlist_id)
    return user_playlist_tracks

# returns the tracks in a playlist in a list of dictionaries
def get_user_playlist_tracks_info(playlist_id):
    user_playlist_tracks = get_user_playlist_tracks(playlist_id)
    user_playlist_tracks_info = []
    for i in range(len(user_playlist_tracks['items'])):
        track = user_playlist_tracks['items'][i]['track']
        return track
        track_info = {
            'track_name': track['name'],
            'track_id': track['id'],
            'track_duration': track['duration_ms'],
            'track_popularity': track['popularity'],
            'track_explicit': track['explicit'],
            'album_name': track['album']['name'],
            'album_id': track['album']['id'],
            'album_release_date': track['album']['release_date'],
            'album_release_date_precision': track['album']['release_date_precision'],
            'album_total_tracks': track['album']['total_tracks'],
            'album_type': track['album']['album_type'],
            'artist_name': track['artists'][0]['name'],
            'artist_id': track['artists'][0]['id']
        }
        user_playlist_tracks_info.append(track_info)
    return user_playlist_tracks_info

# returns a json of recommended tracks according to the seed artists, genres and tracks
def get_recommendations(seed_artists, seed_genres, seed_tracks):
    # returns a json of recommended tracks according to the seed artists, genres and tracks
    recommendations = user_top_read.recommendations(seed_artists=seed_artists, seed_genres=seed_genres, seed_tracks=seed_tracks)
    return recommendations

# get important info from recommendations json
def get_recommendations_info(seed_artists, seed_genres, seed_tracks):
    # get the json of the recommended tracks
    recommendations = get_recommendations(seed_artists, seed_genres, seed_tracks)
    recommendations_info = []
    # get the info of each track
    for i in range(len(recommendations['tracks'])):
        track = recommendations['tracks'][i]
        track_info = get_track_info_from_json(track)
        recommendations_info.append(track_info)
    return recommendations_info

# returns the user's saved albums in a list
def get_saved_albums():
    saved_albums = user_library_read.current_user_saved_albums()
    return saved_albums

# returns the user's saved shows in a list
def get_saved_shows():
    saved_shows = user_library_read.current_user_saved_shows()
    return saved_shows

# returns the user's saved episodes in a list
def get_saved_episodes():
    saved_episodes = user_library_read.current_user_saved_episodes()
    return saved_episodes

# returns the user's library tracks in a list
def get_library_info(limit):
    # get the user's library data
    library = user_library_read.current_user_saved_tracks(limit=limit)
    # convert the library data to a dataframe
    library_df = pd.DataFrame(library)
    return library_df

# Define a function to get the value of a nested key from a dictionary
def get_nested_value(dictionary, keys, default=''):
    # Loop through the keys
    for key in keys:
        # Try to get the value of the current key
        try :
            dictionary = dictionary[key]
        # If the key is not found, return the default value
        except :
            return default
    # Return the final value
    return dictionary

# returns a list of dictionaries containing the user's playlists information
def get_user_playlists_info():
    user_playlists = get_user_playlists()
    user_playlists_info = []
    for i in range(len(user_playlists['items'])):
        playlist = user_playlists['items'][i]
        playlist_info = {
            'playlist_name': playlist['name'],
            'playlist_id': playlist['id'],
            'playlist_description': playlist['description'],
            'playlist_tracks': playlist['tracks']['total'],
            'playlist_owner': playlist['owner']['display_name'],
            'playlist_owner_id': playlist['owner']['id'],
            'playlist_public': playlist['public'],
            'playlist_collaborative': playlist['collaborative'],
        }
        if playlist.get('images'):
            playlist_info['playlist_images'] = playlist['images'][0].get('url', '')
        else:
            playlist_info['playlist_images'] = ''
        # playlist_info['playlist_images'] = playlist.get('images', [{}])[0].get('url', '')

        user_playlists_info.append(playlist_info)
    return user_playlists_info

# store the user's playlists information in a csv file
def store_user_playlists_info():    
    user_playlists_info = get_user_playlists_info()
    df = pd.DataFrame(user_playlists_info)
    # add a column with the current date and time
    df['date'] = datetime.datetime.now()
    df.to_csv('user_playlists_info.csv', index=False)

# get the commands stored in the playlist name or description
def gather_functions(string):
    # lowercases the string
    test = string.lower()
    # write a regex to get the artist which should return swans
    artist = re.search(r'artist-(\w+(?:-\w+)*)', string)
    # write a regex to get the gather which should return gather-all
    gather = re.search(r'gather-(\w+(?:-\w+)*)', string)

    # write a regex to get the genre which should return hip-hop
    genre = re.search(r'genre-(\w+(?:-\w+)*)', string)
    # # write a regex to get the words that has " -" after it
    # words_functions = re.findall(r'(\w+(?:-\w+)*) -', test)
    # dic = {"genre": genre.group(1), "gather": gather.group(1), "artist": artist.group(1)}
    dic = {"genre": None, "gather": None, "artist": None}
    if genre:
        dic["genre"] = genre.group(1).replace("_", " ")
    if gather:
        dic["gather"] = gather.group(1).replace("_", " ")
    if artist:
        dic["artist"] = artist.group(1).replace("_", " ")
    return dic

# get all the playlists that are new
def get_new_playlists():
    
    user_playlist_info = get_user_playlists_info()
    new_df = pd.DataFrame(user_playlist_info)
    new_df['date'] = datetime.datetime.now()
    # read the csv file
    old_df = pd.read_csv('user_playlists_info.csv')

    """Iterate though the new_df and search for matching value from the old dataframe save the index where the first value matches. the value should continue to match."""
    # iterate through the new_df
    user_playlist_info = get_user_playlists_info()
    new_df = pd.DataFrame(user_playlist_info)
    new_df['date'] = datetime.datetime.now()
    # read the csv file
    old_df = pd.read_csv('user_playlists_info.csv')
    the_index = 0
    previous_index = -1
    for i in range(len(new_df)):
        song_id = new_df['playlist_id'][i]
        try:
            # get the index of the matching value
            index = old_df[old_df['playlist_id'] == song_id].index[0]
            # if the index is not the previous index + 1 and it is not the first index
            if i != previous_index+1 and i != 0:
                # save the index
                the_index = i
            previous_index = i
        except IndexError:
            pass
    changed = new_df[:the_index]

    # replace the rows before the index with the new rows, changed
    old_df[:the_index] = changed
    # save the new dataframe to the csv file
    # old_df.to_csv('user_playlists_info.csv', index=False)
    # return all the playlits that are new
    # store_user_playlists_info() # pause while updating the function
    return changed

# get all the playlists with functions hidden in the name or description
def get_playlists_with_functions():
    new_playlists = get_new_playlists()
    # create a dictonary to store the playlists id with functions
    playlists_with_functions = {}
    # iterate through the new playlists
    for i in range(len(new_playlists)):
        # use gather_functions to check if the playlist name and description has functions
        name = gather_functions(new_playlists['playlist_name'][i].lower())
        description = gather_functions(new_playlists['playlist_description'][i].lower())
        funciton_data = {}
        # print(name)
        # print(description)
        # iterate through the dictonary and check if the value is not None
        for key, value in name.items():
            if value:
                # if both the name and description have different values for the same key set a list with both values
                if description[key] and description[key] != value:
                    funciton_data[key] = [value, description[key]]
                else:
                    funciton_data[key] = [value]
            # add playlist id to the dictonary
            # funciton_data['playlist_id'] = new_playlists['playlist_id'][i]
        # save funciton_data to the playlists_with_functions dictonary with key as the playlist id
        playlists_with_functions[new_playlists['playlist_id'][i]] = funciton_data
    return playlists_with_functions

# search for aritst and return the artist id
def search_for_artist(artist):
    # search for the artist
    results = sp.search(q='artist:' + artist, type='artist')
    artist_id = results['artists']['items'][0]['id']
    return artist_id

# get the all the tracks from the artist
def get_artist_discography(name):
    # Get the artist's albums
    results = sp.artist_albums(search_for_artist(name), album_type='album')
    # Extract relevant data from the API response
    all_tarck_ids = []
    for album in results['items']:

        # Get the tracks of the album
        album_tracks = sp.album_tracks(album['id'])

        # Extract relevant data from the API response
        for idx, track in enumerate(album_tracks['items']):
            print(f"Track {idx+1}: {track['name']} by {track['artists'][0]['name']}")
            print(f"Track ID: {track['id']}")
            all_tarck_ids.append(track['id'])
    return all_tarck_ids

def get_friends_activity_json():
    """ get the friends activity using https://github.com/valeriangalliat/spotify-buddylist repository
    the following code runs the node.js get's the friends activity and converts it to json and returns it"""
    friends_activity = subprocess.check_output(["node", r"C:\Users\saket\Documents\GitHub\Pyhton\Project Music\spotify api\spotify-buddylist-master\example.js"])
    # decode the bytes to string
    friends_activity = friends_activity.decode("utf-8")
    friends_activity = json.loads(friends_activity)
    # return the json
    return friends_activity
# get_friends_activity_json()

# write a funciton to store friends activity to a csv file
def store_friends_activity():
    while True:
        # check if there is a friends_activity.csv file if not create one
        try:
            # read the csv file
            friend_activity_csv = pd.read_csv('friend_activity.csv')
        except FileNotFoundError:
            # create a csv file
            with open('friend_activity.csv', 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['user_uri', 'track_uri', 'timestamp', 'current_time'])
        try:
            # read the csv file
            friends_activity_csv = pd.read_csv('friends_activity.csv')
        except FileNotFoundError:
            # create a csv file
            with open('friends_activity.csv', 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['user_uri', 'track_uri', 'timestamp', 'current_time'])
        staring_time = datetime.datetime.now()
        
        # get the friends activity
        friends_activity_json = get_friends_activity_json()
        # get the current time
        current_time = datetime.datetime.now()
        # iterate through the friends activity

        user_ids = ["gntab9tp1cc5qipthodlvvsm3"]
        user_id = ["spotify:user:"+user_id for user_id in user_ids]
        try:
            for friend in friends_activity_json['friends']:
                    # check if the user uri is the same as the user uri
                    if friend['user']['uri'] in user_id:        # only for some selected users
                        # check if the track is same as the previous track don't add it to the csv file
                        """ This will run if the csv file is empty or if the track is not the same as the previous track"""
                        try:
                            if friend['track']['uri'] == friend_activity_csv['track_uri'].iloc[-1]:
                                continue
                        except IndexError:
                            pass

                        # write the data to the csv file
                        with open('friend_activity.csv', 'a', newline='') as file:
                            writer = csv.writer(file)
                            print('adding to the database', friend['user']['uri'], friend['track']['uri'], friend['timestamp'], current_time)
                            writer.writerow([friend['user']['uri'], friend['track']['uri'], friend['timestamp'], current_time])

                        '''Time to add the track to the playlist if it is not already in the playlist '''
                        playlist_id = "5XV9floz2zzeWiAcduWBkc"
                        # add the track to the private playlist if it is not already in the playlist
                        if friend['track']['uri'].split(":")[-1] not in [i['track']['uri'].split(":")[-1] for i in get_playlist_tracks(playlist_id)]:
                            # add the track to the playlist
                            song_id = friend['track']['uri'].split(":")[-1]
                            print(song_id)
                            # print song name
                            print(friend['track']['name'])
                            sp.user_playlist_add_tracks(username, playlist_id, [song_id])

                        
                        """trying to play the track"""
                        # sp.start_playback(uris=[friend['track']['uri']])
                        # add the track to queue
                        # sp.add_to_queue(friend['track']['uri'])
            
            # run this code every 30 minutes using start_time and reset the start_time
            if (datetime.datetime.now() - staring_time).seconds > 1800:
                print('30 hour is over and updating the friends_activity.csv file')
                """ I wan't able to update only if new data is added to the csv file so I am just updating the csv file every 30 minutes"""
                
                # write the data to the csv file
                for friend in friends_activity_json['friends']:
                    with open('friends_activity.csv', 'a', newline='') as file:
                        writer = csv.writer(file)
                        writer.writerow([friend['user']['uri'], friend['track']['uri'], friend['timestamp'], current_time])
                # reset the staring time
                staring_time = datetime.datetime.now()
            # sleep for 20 seconds
            print('sleeping for 30 sec')
            for i in range(30):
                print('searching for friends activity in', i, 'sec')
                time.sleep(1)
            # time.sleep(60)
            # break
        except KeyError:
            print('KeyError')
            time.sleep(30)

if __name__ == "__main__":
    store_friends_activity()

adding to the database spotify:user:gntab9tp1cc5qipthodlvvsm3 spotify:track:0dYOhXotUJibq5j61vZQVg 1682094424181 2023-04-21 21:59:35.149572
0dYOhXotUJibq5j61vZQVg
Disappear
sleeping for 30 sec
searching for friends activity in 0 sec
searching for friends activity in 1 sec
searching for friends activity in 2 sec
searching for friends activity in 3 sec
searching for friends activity in 4 sec
searching for friends activity in 5 sec
searching for friends activity in 6 sec
searching for friends activity in 7 sec
searching for friends activity in 8 sec
searching for friends activity in 9 sec
searching for friends activity in 10 sec
searching for friends activity in 11 sec
searching for friends activity in 12 sec
searching for friends activity in 13 sec
searching for friends activity in 14 sec
searching for friends activity in 15 sec
searching for friends activity in 16 sec
searching for friends activity in 17 sec
searching for friends activity in 18 sec
searching for friends activity in 19 s

CalledProcessError: Command '['node', 'C:\\Users\\saket\\Documents\\GitHub\\Pyhton\\Project Music\\spotify api\\spotify-buddylist-master\\example.js']' returned non-zero exit status 1.

In [ ]:

def get_friend_activity_json():
    try:
    # read the csv file
        friend_activity_csv = pd.read_csv('friend_activity.csv')
    except FileNotFoundError:
        # create a csv file
        with open('friend_activity.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['user_uri', 'track_uri', 'timestamp', 'current_time'])
    user_ids = ["gntab9tp1cc5qipthodlvvsm3"]
    user_id = ["spotify:user:"+user_id for user_id in user_ids]

    current_time = datetime.datetime.now()

    for friend in friends_activity_json['friends']:
            # check if the user uri is the same as the user uri
            if friend['user']['uri'] in user_id:        # only for some selected users
                # check if the track is same as the previous track don't add it to the csv file
                """ This will run if the csv file is empty or if the track is not the same as the previous track"""
                try:
                    if friend['track']['uri'] == friend_activity_csv['track_uri'].iloc[-1]:
                        continue
                except IndexError:
                    pass

                # write the data to the csv file
                with open('friend_activity.csv', 'a', newline='') as file:
                    writer = csv.writer(file)
                    print('adding to the database', friend['user']['uri'], friend['track']['uri'], friend['timestamp'], current_time)
                    writer.writerow([friend['user']['uri'], friend['track']['uri'], friend['timestamp'], current_time])

                '''Time to add the track to the playlist if it is not already in the playlist '''
                playlist_id = "5XV9floz2zzeWiAcduWBkc"
                # add the track to the private playlist if it is not already in the playlist
                if friend['track']['uri'].split(":")[-1] not in [i['track']['uri'].split(":")[-1] for i in get_playlist_tracks(playlist_id)]:
                    # add the track to the playlist
                    song_id = friend['track']['uri'].split(":")[-1]
                    print(song_id)
                    sp.user_playlist_add_tracks(username, playlist_id, [song_id])


In [1]:


# write a funciton to store friends activity to a csv file
def store_friends_activity():
    playlist_id = "5XV9floz2zzeWiAcduWBkc"
    while True:
        # check if there is a friends_activity.csv file if not create one
        try:
            # read the csv file
            friend_activity_csv = pd.read_csv('friend_activity.csv')
        except FileNotFoundError:
            # create a csv file
            with open('friend_activity.csv', 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['user_uri', 'track_uri', 'timestamp', 'current_time'])
        try:
            # read the csv file
            friends_activity_csv = pd.read_csv('friends_activity.csv')
        except FileNotFoundError:
            # create a csv file
            with open('friends_activity.csv', 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['user_uri', 'track_uri', 'timestamp', 'current_time'])
        staring_time = datetime.datetime.now()
        
        # get the friends activity
        friends_activity_json = get_friends_activity_json()
        # get the current time
        current_time = datetime.datetime.now()
        # iterate through the friends activity
                        
        user_ids = ["gntab9tp1cc5qipthodlvvsm3"]
        user_id = ["spotify:user:"+user_id for user_id in user_ids]

        for friend in friends_activity_json['friends']:
                # check if the user uri is the same as the user uri
                if friend['user']['uri'] in user_id:        # only for some selected users
                    # check if the track is same as the previous track don't add it to the csv file
                    """ This will run if the csv file is empty or if the track is not the same as the previous track"""
                    try:
                        if friend['track']['uri'] == friend_activity_csv['track_uri'].iloc[-1]:
                            continue
                    except IndexError:
                        pass

                    # write the data to the csv file
                    with open('friend_activity.csv', 'a', newline='') as file:
                        writer = csv.writer(file)
                        print('adding to the database', friend['user']['uri'], friend['track']['uri'], friend['timestamp'], current_time)
                        writer.writerow([friend['user']['uri'], friend['track']['uri'], friend['timestamp'], current_time])

                    '''Time to add the track to the playlist if it is not already in the playlist '''
                    playlist_id = "5XV9floz2zzeWiAcduWBkc"
                    # add the track to the private playlist if it is not already in the playlist
                    if friend['track']['uri'].split(":")[-1] not in [i['track']['uri'].split(":")[-1] for i in get_playlist_tracks(playlist_id)]:
                        # add the track to the playlist
                        song_id = friend['track']['uri'].split(":")[-1]
                        print(song_id)
                        sp.user_playlist_add_tracks(username, playlist_id, [song_id])

                    
                    """trying to play the track"""
                    # sp.start_playback(uris=[friend['track']['uri']])
                    # add the track to queue
                    # sp.add_to_queue(friend['track']['uri'])
        
        # run this code every 30 minutes using start_time and reset the start_time
        if (datetime.datetime.now() - staring_time).seconds > 1800:
            print('30 hour is over and updating the friends_activity.csv file')
            """ I wan't able to update only if new data is added to the csv file so I am just updating the csv file every 30 minutes"""
            
            # write the data to the csv file
            for friend in friends_activity_json['friends']:
                with open('friends_activity.csv', 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([friend['user']['uri'], friend['track']['uri'], friend['timestamp'], current_time])
            # reset the staring time
            staring_time = datetime.datetime.now()
        # sleep for 20 seconds
        time.sleep(20)
        # break
store_friends_activity()

NameError: name 'pd' is not defined

In [129]:
playlist = 

[{'added_at': '2023-04-20T23:54:10Z',
  'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/rt47etgc6xpwhhhb8575rth83'},
   'href': 'https://api.spotify.com/v1/users/rt47etgc6xpwhhhb8575rth83',
   'id': 'rt47etgc6xpwhhhb8575rth83',
   'type': 'user',
   'uri': 'spotify:user:rt47etgc6xpwhhhb8575rth83'},
  'is_local': False,
  'primary_color': None,
  'track': {'album': {'album_group': 'album',
    'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02'},
      'href': 'https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02',
      'id': '06HL4z0CvFAxyc27GXpf02',
      'name': 'Taylor Swift',
      'type': 'artist',
      'uri': 'spotify:artist:06HL4z0CvFAxyc27GXpf02'}],
    'available_markets': ['AD',
     'AE',
     'AG',
     'AL',
     'AM',
     'AO',
     'AR',
     'AT',
     'AU',
     'AZ',
     'BA',
     'BB',
     'BD',
     'BE',
     'BF',
     'BG',
     'BH',
     'BI',
     'BJ'

['1wtOxkiel43cVs0Yux5Q4h',
 '6pS0PVQYaBYGkKPRxT5PEl',
 '0eVz3hV2xOXdneGpnWDFpb',
 '126iSnNOjQBhk8fEosNnVD',
 '51HiIbv8gO0HvwpmAxXAFe',
 '0eBryM7ePQH3Klt3jz8xZd',
 '2Mkon0rfeN6GRFWv5Ag1NC',
 '1wXJjKJy5Pt38Bu3tUzTUL',
 '1I3f8tsoYbIPEgQ1TPvJPb',
 '0bHpsorTpt9wX6Dkr2LEfg',
 '2dp8i0NgWI7dbCtPNC1Igc',
 '1Tii3aXrNzb8IDXs6cxfZz',
 '77zsHdb523rUQnOYUSG3qb',
 '5aV4HUW9RFOB0aXq0Ud9s0',
 '3GXSywNvYLAVUCtjMHkKDh',
 '20I6sIOMTCkB6w7ryavxtO',
 '20I6sIOMTCkB6w7ryavxtO',
 '20I6sIOMTCkB6w7ryavxtO']

In [136]:
for i in playlist:
    print(i)

1wtOxkiel43cVs0Yux5Q4h
6pS0PVQYaBYGkKPRxT5PEl
0eVz3hV2xOXdneGpnWDFpb
126iSnNOjQBhk8fEosNnVD
51HiIbv8gO0HvwpmAxXAFe
0eBryM7ePQH3Klt3jz8xZd
2Mkon0rfeN6GRFWv5Ag1NC
1wXJjKJy5Pt38Bu3tUzTUL
1I3f8tsoYbIPEgQ1TPvJPb
0bHpsorTpt9wX6Dkr2LEfg
2dp8i0NgWI7dbCtPNC1Igc
1Tii3aXrNzb8IDXs6cxfZz
77zsHdb523rUQnOYUSG3qb
5aV4HUW9RFOB0aXq0Ud9s0
3GXSywNvYLAVUCtjMHkKDh
20I6sIOMTCkB6w7ryavxtO
20I6sIOMTCkB6w7ryavxtO
20I6sIOMTCkB6w7ryavxtO


In [123]:
song_id = '20I6sIOMTCkB6w7ryavxtO'
playlist_id = "5XV9floz2zzeWiAcduWBkc"
# add_to_playlist(playlist_id, song_id)

# add song to the playlist
def add_to_playlist(playlist_id, song_id):
    # add the track to the playlist
    print('added to the playlist')

add_to_playlist(playlist_id, song_id)

added to the playlist


In [43]:
def store_friends_activity():
    # check if there is a friends_activity.csv file if not create one
    try:
        # read the csv file
        friends_activity = pd.read_csv('friends_activity.csv')
    except FileNotFoundError:
        # create a csv file
        # with open('friends_activity.csv', 'w', newline='') as file:
            # writer = csv.writer(file)
            # iterate through the friends activity and set each friend uri as column names
        friends_activity_json = get_friends_activity_json()
        friend_uris = []
        for friend in friends_activity_json['friends']:
            friend_uri = friend['user']['uri'].split(':')[2]
            friend_uris.append(friend_uri)

store_friends_activity()

In [18]:

    # print(friends_activity)
    # iterate through the friends activity
    for friend in friends_activity['friends']:
    
    user_ids = ["gntab9tp1cc5qipthodlvvsm3"]
    user_id = ["spotify:user:"+user_id for user_id in user_ids]

        # check if the user uri is the same as the user uri
        if friend['user']['uri'] in user_id:        # only for some selected users
            pass
friends_activity = subprocess.check_output(["node", r"C:\Users\saket\Documents\GitHub\Pyhton\Project Music\spotify api\spotify-buddylist-master\example.js"])


In [28]:

# get the friends activity
friends_activity = subprocess.check_output(["node", r"C:\Users\saket\Documents\GitHub\Pyhton\Project Music\spotify api\spotify-buddylist-master\example.js"])

# decode the bytes to string
friends_activity = friends_activity.decode("utf-8")
friends_activity = json.loads(friends_activity)

user_ids = ["gntab9tp1cc5qipthodlvvsm3"]
user_id = ["spotify:user:"+user_id for user_id in user_ids]
# iterate through the friends activity
for friend in friends_activity['friends']:
    # check if the user uri is the same as the user uri
    if friend['user']['uri'] in user_id:
        print(friend['user']['name'], 'playing ......')
        # print the name of the person
    print(friend['track']['name'], friend['user']['name'])

Wim ari
New Drugs Ken Langelier
Hand In Hand Eliott
Ohms ARTV
Apocalypse Irles Music
VIRTUAL BIRDS 43giyotin
Raspberry Fields Miles Lambert-Gilliam
ふたり maja
Hell N Back Topsify
Waffles madeline
Falsafa - From "Dastaan" gaurisha
Baby, I Had an Abortion HuzgOnline


In [33]:
food_amount = float(input("enter the amount of food: "))
tip_percentage = float(input("enter the tip percentage: "))/100
tip_amout = food_amount * tip_percentage
total_amount = food_amount + tip_amout
print("the total amount is: ", total_amount)


the total amount is:  130.0


In [30]:
''' try to sture all the user activity in a json file '''

import json

# define the new track and timestamp as a dictionary
new_track = get_friends_activity_json()

friends_activity_json = get_friends_activity_json()

# get the new track and timestamp from the user

# open the json file in read mode and load its content into a dictionary
with open('friends_activity.json', 'r') as f:
  friends_activity = json.load(f)

# find the index of user1 in the friends list
user1_index = -1
for i, friend in enumerate(friends_activity_json['friends']):
  if friend['user']['uri'] == 'user1':
    user1_index = i
    break

# if user1 is found, update the track and timestamp values
if user1_index != -1:
  friends_activity['friends'][user1_index]['track'] = new_track['track']
  friends_activity['friends'][user1_index]['timestamp'] = new_track['timestamp']

# open the json file in write mode and overwrite its content with the updated dictionary
with open('friends_activity.json', 'w') as f:
  json.dump(friends_activity, f)

{'friends': [{'timestamp': 1681932894237,
   'user': {'uri': 'spotify:user:rex1hi2s1ay9q7hf3km0v9qab',
    'name': 'ari',
    'imageUrl': 'https://i.scdn.co/image/ab67757000003b829b2a2b171446edbd4e4672bb'},
   'track': {'uri': 'spotify:track:3DpGkTydxKfJ8vcWobZ7yi',
    'name': 'Wim',
    'imageUrl': 'http://i.scdn.co/image/ab67616d0000b27300b9e955d2b2a0be08152a8c',
    'album': {'uri': 'spotify:album:1lf3rLmrO0ObMieg4YqEuB',
     'name': 'Colour Green'},
    'artist': {'uri': 'spotify:artist:2ckfYpz65ZueWZa2dB6yXC',
     'name': 'Sibylle Baier'},
    'context': {'uri': 'spotify:album:68kIBfvmlkUHmzJyZbiH5Z',
     'name': 'Let Me Go, Let Me Go, Let Me Go',
     'index': 0}}},
  {'timestamp': 1681939645675,
   'user': {'uri': 'spotify:user:12160373356',
    'name': 'Ken Langelier',
    'imageUrl': 'https://scontent-ord5-2.xx.fbcdn.net/v/t31.18172-1/271383_10150255226814398_3854314_o.jpg?stp=c4.4.49.49a_cp0_dst-jpg_p57x57&_nc_cat=107&ccb=1-7&_nc_sid=dbb9e7&_nc_ohc=6Ez8LI604OsAX_W2GmO&_nc

In [23]:
# friends_activity = friends_activity.decode("utf-8")
# friends_activity = json.loads(friends_activity)

for i in friends_activity['friends']:
    print(i['user']['name'])
    # print(i['user']['uri'])
    # print(i['track']['name'])
    # print(i['timestamp'])

ari
Ken Langelier
Eliott
ARTV
Irles Music
43giyotin
Miles Lambert-Gilliam
maja
Topsify
madeline
HuzgOnline
gaurisha


In [ ]:
import json
friends_activity = json.loads(friends_activity)


In [70]:
for i in friends_activity['friends']:
    print(i['']['name'])

gaurisha
ari
Ken Langelier
Eliott
ARTV
HuzgOnline
Irles Music
suchita
madeline
43giyotin
Topsify
maja
Miles Lambert-Gilliam


In [8]:
friends_activity['friends']

[{'timestamp': 1681806866444,
  'user': {'uri': 'spotify:user:02aj59ugjm84jmkqaimbjl2hb',
   'name': 'gaurisha',
   'imageUrl': 'https://i.scdn.co/image/ab67757000003b82bf61348dee9bcfd870474dab'},
  'track': {'uri': 'spotify:track:4gCVaEUubhHmhsEe0GFRVx',
   'name': 'Tuta Pull Wahan',
   'imageUrl': 'http://i.scdn.co/image/ab67616d0000b27385cf9fb0cb24b7895dbfe8da',
   'album': {'uri': 'spotify:album:2gFMOqvZHiCwD6myI65Okz',
    'name': 'Tuta Pull Wahan'},
   'artist': {'uri': 'spotify:artist:1VFIeAFbzMVHmvGQtfE6OI',
    'name': 'Deepak Rathore Project'},
   'context': {'uri': 'spotify:album:2gFMOqvZHiCwD6myI65Okz',
    'name': 'Tuta Pull Wahan',
    'index': 0}}},
 {'timestamp': 1681932894237,
  'user': {'uri': 'spotify:user:rex1hi2s1ay9q7hf3km0v9qab',
   'name': 'ari',
   'imageUrl': 'https://i.scdn.co/image/ab67757000003b829b2a2b171446edbd4e4672bb'},
  'track': {'uri': 'spotify:track:3DpGkTydxKfJ8vcWobZ7yi',
   'name': 'Wim',
   'imageUrl': 'http://i.scdn.co/image/ab67616d0000b27300b

In [31]:
# arrage the playlist by the given autio feature
def arrage_by_popularity(playlist_id):
    # get the playlist
    playlist = sp.playlist(playlist_id)
    # get the tracks from the playlist
    tracks = playlist['tracks']['items']
    # get the track ids
    track_ids = [track['track']['id'] for track in tracks]
    # get the audio features for the tracks
    features = sp.audio_features(track_ids)
    # create a dataframe with the audio features
    df = pd.DataFrame(features)
    # sort the dataframe by the given feature
    df.sort_values(by=popularity, inplace=True)
    # get the track ids from the sorted dataframe
    sorted_track_ids = df['id'].tolist()
    # create a new playlist
    new_playlist = sp.user_playlist_create(user_id, f"{playlist['name']} - {popularity}")
    # add the tracks to the new playlist
    sp.user_playlist_add_tracks(user_id, new_playlist['id'], sorted_track_ids)

In [ ]:
arrage_by_popularity('')

In [30]:
song = sp.track('1BqojSE4r293ccJ5TIFQDk')

In [28]:
{
  "friends": [
    {
      "timestamp": 1681806866444,
      "user": {
        "uri": "spotify:user:02aj59ugjm84jmkqaimbjl2hb",
        "name": "gaurisha",
        "imageUrl": "https://i.scdn.co/image/ab67757000003b82bf61348dee9bcfd870474dab"
      },
      "track": {
        "uri": "spotify:track:4gCVaEUubhHmhsEe0GFRVx",
        "name": "Tuta Pull Wahan",
        "imageUrl": "http://i.scdn.co/image/ab67616d0000b27385cf9fb0cb24b7895dbfe8da",
        "album": {
          "uri": "spotify:album:2gFMOqvZHiCwD6myI65Okz",
          "name": "Tuta Pull Wahan"
        },
        "artist": {
          "uri": "spotify:artist:1VFIeAFbzMVHmvGQtfE6OI",
          "name": "Deepak Rathore Project"
        },
        "context": {
          "uri": "spotify:album:2gFMOqvZHiCwD6myI65Okz",
          "name": "Tuta Pull Wahan",
          "index": 0
        }
      }
    },
    {
      "timestamp": 1681843767387,
      "user": {
        "uri": "spotify:user:topsify",
        "name": "Topsify",
        "imageUrl": "https://i.scdn.co/image/ab67757000003b82ad2b502872293bac301b8fd6"
      },
      "track": {
        "uri": "spotify:track:40x8vMUHIjqd2A5EtHEvAb",
        "name": "Drowning - Sped Up Version",
        "imageUrl": "http://i.scdn.co/image/ab67616d0000b273e1f6ae883db35c45ba55f016",
        "album": {
          "uri": "spotify:album:3Z5FPyxx1sKOTzHsOnS4ZI",
          "name": "Drowning (Sped Up Version)"
        },
        "artist": {
          "uri": "spotify:artist:0M2CO5ijP35MDhNwvpgxTV",
          "name": "sped up nightcore"
        },
        "context": {
          "uri": "spotify:playlist:7sRtL4EdETQwjmP4CQcP9Z",
          "name": "sped up songs ⚡️ fast versions of tiktok trends + viral hits",
          "index": 0
        }
      }
    },
    {
      "timestamp": 1681865794427,
      "user": {
        "uri": "spotify:user:xbqlbum9p8gknixdovte1d15c",
        "name": "43giyotin",
        "imageUrl": "https://i.scdn.co/image/ab67757000003b82850b9f39868e881264d4ba7d"
      },
      "track": {
        "uri": "spotify:track:5LON6xrKNqaFWVwI4HrazZ",
        "name": "guardianAngels",
        "imageUrl": "http://i.scdn.co/image/ab67616d0000b273b442eeb09f3d02dc74634b79",
        "album": {
          "uri": "spotify:album:4O5mMyYDzbj5Zey0nslX6A",
          "name": "guardianAngels"
        },
        "artist": {
          "uri": "spotify:artist:1NZYxkl40ebW2QiwkMctIY",
          "name": "Drain Gang Archive"
        },
        "context": {
          "uri": "spotify:album:4O5mMyYDzbj5Zey0nslX6A",
          "name": "guardianAngels",
          "index": 0
        }
      }
    },
    {
      "timestamp": 1681900671593,
      "user": {
        "uri": "spotify:user:jaime33",
        "name": "Irles Music",
        "imageUrl": "https://i.scdn.co/image/ab67757000003b82cdbf4610bef6efb87adb4daf"
      },
      "track": {
        "uri": "spotify:track:1UZxvWYq91lf5fpaQn8DOv",
        "name": "Hello",
        "imageUrl": "http://i.scdn.co/image/ab67616d0000b273ca89ff8d60c33394970c3bdc",
        "album": {
          "uri": "spotify:album:5GcGGnuWS9RnSKV8mg3NIT",
          "name": "Hello"
        },
        "artist": {
          "uri": "spotify:artist:0RkvwB5IWAviv0qhBsB3GW",
          "name": "KöTTBULLAR"
        },
        "context": {
          "uri": "spotify:album:5GcGGnuWS9RnSKV8mg3NIT",
          "name": "Hello",
          "index": 0
        }
      }
    },
    {
      "timestamp": 1681920313706,
      "user": {
        "uri": "spotify:user:miles_lamborgenie",
        "name": "Miles Lambert-Gilliam",
        "imageUrl": "https://i.scdn.co/image/ab67757000003b82656acdcdafa5afe6146ee695"
      },
      "track": {
        "uri": "spotify:track:7E0zDVUHKmBVuckXrliBwK",
        "name": "One Eleven",
        "imageUrl": "http://i.scdn.co/image/ab67616d0000b27340dcfbf3ef5d494ec137e9a2",
        "album": {
          "uri": "spotify:album:7xJ7jHNu3JNfdnao9xwMho",
          "name": "Donuts"
        },
        "artist": {
          "uri": "spotify:artist:0IVcLMMbm05VIjnzPkGCyp",
          "name": "J Dilla"
        },
        "context": {
          "uri": "spotify:playlist:1CRXsDT9gAPRt2wO5QyssS",
          "name": "Tᕼᕮ ᗷIG ᑭIᑕTᑌᖇᕮ",
          "index": 0
        }
      }
    },
    {
      "timestamp": 1681931153973,
      "user": {
        "uri": "spotify:user:earm28v0wpfycco58h1e64qi4",
        "name": "HuzgOnline",
        "imageUrl": "https://i.scdn.co/image/ab67757000003b8243ad9adbb38d891c9c34b214"
      },
      "track": {
        "uri": "spotify:track:4zSmsHur194jO2hGCTt6Hn",
        "name": "Superstar",
        "imageUrl": "http://i.scdn.co/image/ab67616d0000b273e08b1250db5f75643f1508c9",
        "album": {
          "uri": "spotify:album:1BZoqf8Zje5nGdwZhOjAtD",
          "name": "The Miseducation of Lauryn Hill"
        },
        "artist": {
          "uri": "spotify:artist:2Mu5NfyYm8n5iTomuKAEHl",
          "name": "Ms. Lauryn Hill"
        },
        "context": {
          "uri": "spotify:album:1BZoqf8Zje5nGdwZhOjAtD",
          "name": "The Miseducation of Lauryn Hill",
          "index": 0
        }
      }
    },
    {
      "timestamp": 1681932894237,
      "user": {
        "uri": "spotify:user:rex1hi2s1ay9q7hf3km0v9qab",
        "name": "ari",
        "imageUrl": "https://i.scdn.co/image/ab67757000003b829b2a2b171446edbd4e4672bb"
      },
      "track": {
        "uri": "spotify:track:3DpGkTydxKfJ8vcWobZ7yi",
        "name": "Wim",
        "imageUrl": "http://i.scdn.co/image/ab67616d0000b27300b9e955d2b2a0be08152a8c",
        "album": {
          "uri": "spotify:album:1lf3rLmrO0ObMieg4YqEuB",
          "name": "Colour Green"
        },
        "artist": {
          "uri": "spotify:artist:2ckfYpz65ZueWZa2dB6yXC",
          "name": "Sibylle Baier"
        },
        "context": {
          "uri": "spotify:album:68kIBfvmlkUHmzJyZbiH5Z",
          "name": "Let Me Go, Let Me Go, Let Me Go",
          "index": 0
        }
      }
    },
    {
      "timestamp": 1681939645675,
      "user": {
        "uri": "spotify:user:12160373356",
        "name": "Ken Langelier",
        "imageUrl": "https://scontent-ord5-2.xx.fbcdn.net/v/t31.18172-1/271383_10150255226814398_3854314_o.jpg?stp=c4.4.49.49a_cp0_dst-jpg_p57x57&_nc_cat=107&ccb=1-7&_nc_sid=dbb9e7&_nc_ohc=6Ez8LI604OsAX9CZ8Uj&_nc_ht=scontent-ord5-2.xx&edm=AP4hL3IEAAAA&oh=00_AfA7Qfy_YFD3S3rtA0T5J5DzlB1KAb1f5qExpr996H2_Ww&oe=64681462"
      },
      "track": {
        "uri": "spotify:track:3eV2rBvsjBxJQRWcvkI7bB",
        "name": "New Drugs",
        "imageUrl": "http://i.scdn.co/image/ab67616d0000b273812c8f4ea9495edeb6989b5c",
        "album": {
          "uri": "spotify:album:6oavlRWmepUQz1tJLfD3fY",
          "name": "The Bottom"
        },
        "artist": {
          "uri": "spotify:artist:0gBm2aEXTuXG0y5cctBIO7",
          "name": "Dead When I Found Her"
        },
        "context": {
          "uri": "spotify:playlist:3utBOus8vmn1l3hwBKt1hC",
          "name": "Langeleviver's Alternative",
          "index": 0
        }
      }
    },
    {
      "timestamp": 1681945398140,
      "user": {
        "uri": "spotify:user:gntab9tp1cc5qipthodlvvsm3",
        "name": "suchita",
        "imageUrl": "https://i.scdn.co/image/ab67757000003b829dc4867ae0babe8b2d882f5a"
      },
      "track": {
        "uri": "spotify:track:3uCkIqD0VzQUijbs8WIizs",
        "name": "These Days",
        "imageUrl": "http://i.scdn.co/image/ab67616d0000b273b1235c0092f6658466e96819",
        "album": {
          "uri": "spotify:album:6O62Cqi0YqOytEbFmeLyjU",
          "name": "Chelsea Girl"
        },
        "artist": {
          "uri": "spotify:artist:0IwlY33zbBXN7zlS9DP2Cj",
          "name": "Nico"
        },
        "context": {
          "uri": "spotify:album:6O62Cqi0YqOytEbFmeLyjU",
          "name": "Chelsea Girl",
          "index": 0
        }
      }
    },
    {
      "timestamp": 1681958160201,
      "user": {
        "uri": "spotify:user:blbtthk7vp3avrm1hb49felpw",
        "name": "madeline",
        "imageUrl": "https://i.scdn.co/image/ab67757000003b82942f1920d42aa4c5df176de4"
      },
      "track": {
        "uri": "spotify:track:4w9soAM7IrmYDhSXLp14p6",
        "name": "Peaches",
        "imageUrl": "http://i.scdn.co/image/ab67616d0000b273c21aa4015cb02b6b8eca09a4",
        "album": {
          "uri": "spotify:album:3QITNyRzvTqn0t9tLmS1Vd",
          "name": "The Super Mario Bros. Movie (Original Motion Picture Soundtrack)"
        },
        "artist": {
          "uri": "spotify:artist:0qpMYTgbXRi1ZcX6vend3T",
          "name": "Jack Black"
        },
        "context": {
          "uri": "spotify:album:3QITNyRzvTqn0t9tLmS1Vd",
          "name": "The Super Mario Bros. Movie (Original Motion Picture Soundtrack)",
          "index": 0
        }
      }
    },
    {
      "timestamp": 1681967089803,
      "user": {
        "uri": "spotify:user:homestar99",
        "name": "Eliott",
        "imageUrl": "https://i.scdn.co/image/ab67757000003b8233e21b088263fa0f21ac5f2b"
      },
      "track": {
        "uri": "spotify:track:7bxAniEn5wxHmEfJZkD9dg",
        "name": "Hand In Hand",
        "imageUrl": "http://i.scdn.co/image/ab67616d0000b273121c38d2ab42b64a7ec09f9e",
        "album": {
          "uri": "spotify:album:0mUFefHSr0Ovi9vNcUGppt",
          "name": "This Year's Model"
        },
        "artist": {
          "uri": "spotify:artist:4qmHkMxr6pTWh5Zo74odpH",
          "name": "Elvis Costello & The Attractions"
        },
        "context": {
          "uri": "spotify:playlist:4RSutFssm5BrHALNngF9hf",
          "name": "That's Why God Made the Radio",
          "index": 0
        }
      }
    },
    {
      "timestamp": 1681973618216,
      "user": {
        "uri": "spotify:user:compton182",
        "name": "ARTV",
        "imageUrl": "https://i.scdn.co/image/ab67757000003b82c37f6298191d13be6ed2d158"
      },
      "track": {
        "uri": "spotify:track:0h6vLT1rLneoc5I4sOxvB1",
        "name": "Ohms",
        "imageUrl": "http://i.scdn.co/image/ab67616d0000b27366c9f5d6f13bfc9abedc1056",
        "album": {
          "uri": "spotify:album:0VEFy5MsBiq0u2lWL0OwOd",
          "name": "Ohms"
        },
        "artist": {
          "uri": "spotify:artist:6Ghvu1VvMGScGpOUJBAHNH",
          "name": "Deftones"
        },
        "context": {
          "uri": "spotify:album:0VEFy5MsBiq0u2lWL0OwOd",
          "name": "Ohms",
          "index": 0
        }
      }
    },
    {
      "timestamp": 1681974783645,
      "user": {
        "uri": "spotify:user:13v0l2zagkat9thdoqngm8ki8",
        "name": "maja",
        "imageUrl": "https://i.scdn.co/image/ab67757000003b822630b0839d17ab28f6be5b99"
      },
      "track": {
        "uri": "spotify:track:6t8qLHHisTXTNvkURTxNlM",
        "name": "Empty Stage",
        "imageUrl": "http://i.scdn.co/image/ab67616d0000b273bb0e6d398aee4b24e7a4acf8",
        "album": {
          "uri": "spotify:album:0NrIhUxqXHbIR75LWZNURP",
          "name": "At First Light"
        },
        "artist": {
          "uri": "spotify:artist:1tbxopipOkDjnRDfMKj535",
          "name": "Ralph Towner"
        },
        "context": {
          "uri": "spotify:album:3W9EXpRzxTBKORutMEamev",
          "name": "Les Égarés",
          "index": 0
        }
      }
    }
  ]
}

album
artists
available_markets
disc_number
duration_ms
explicit
external_ids
external_urls
href
id
is_local
name
popularity
preview_url
track_number
type
uri


In [12]:
arrange_playlist_by_feature('06baNHXoFp1H7XnwWk2bBS', 'popularity')

1BqojSE4r293ccJ5TIFQDk
{'danceability': 0.247, 'energy': 0.169, 'key': 2, 'loudness': -23.395, 'mode': 1, 'speechiness': 0.0354, 'acousticness': 0.803, 'instrumentalness': 0.882, 'liveness': 0.11, 'valence': 0.237, 'tempo': 77.227, 'type': 'audio_features', 'id': '1BqojSE4r293ccJ5TIFQDk', 'uri': 'spotify:track:1BqojSE4r293ccJ5TIFQDk', 'track_href': 'https://api.spotify.com/v1/tracks/1BqojSE4r293ccJ5TIFQDk', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1BqojSE4r293ccJ5TIFQDk', 'duration_ms': 120256, 'time_signature': 3}


NameError: name 'modify_private_playlist' is not defined

In [5]:

def get_artist_top_track(id):
    # get the artist's all albums
    album_tracks = sp.artist_albums(id, album_type='album')

    # iterate through the albums
    for album in album_tracks['items']:
        print(album['name'])

swans = get_artist_top_track(search_for_artist('swans'))
print(swans)


leaving meaning.
The Glowing Man
Love of Life
To Be Kind
The Seer
My Father Will Guide Me Up a Rope to the Sky
Swans Are Dead: Live '95-'97
Children of God / World of Skin
Filth / Body to Body, Job to Job
Various Failures
Soundtracks for the Blind
Die Tür Ist Zu
The Great Annihilator (Remastered)
Great Annihilator
Love of Life (Deluxe Edition)
White Light from the Mouth of Infinity
Feel Good Now (2020 Remaster)
Children of God
Public Castration Is a Good Idea (Live)
Greed / Time Is Money (Bastard)
None


In [24]:
add_to_playlist("06baNHXoFp1H7XnwWk2bBS", all_tarck_ids)

len -  4


In [ ]:
get_artist_discography(get_artist_id('swans'))



In [26]:
# execute the function from the playlist
def execute_function():
    # get playlists with functions
    playlists_with_functions = get_playlists_with_functions()
    # iterate through the playlists_with_functions
    for playlist_id, function_data in playlists_with_functions.items():
        # iterate through the function_data
        # for key as aritst
        if 'artist' in function_data:
            # iterate through the values
            for artist in function_data['artist']:
                if 'gather' not in function_data:
                    # search for the artist
                    artist_id = search_for_artist(artist)
                    # get all the songs from the artist
                    results = all_tarck_ids
                    add_to_playlist(playlist_id, results)
                    
execute_function()

['1BqojSE4r293ccJ5TIFQDk', '6BaVV2W46KUrMUSl4JH1RA', '53clfBEivykFfpajRakKA1', '1Jx5nuSNFZ4dmGSPYxCI6R', '7elg4fjqBkSvoFlDlXL5pk', '5cZXZJgQWFGl0BMfNGJTRw', '7yNakEyU0xkgD17UodjtXv', '3nn4Iko6zFPL19kwWfcT6A', '6mDD8dnYp25mRbupOPBLL6', '6LjFUzIP3pPI5Lh4a8FWxi', '0JMgNPZ60wOBNbyOMHuJTZ', '5zjxoc982f9VVHvRXcO3kv', '2M3Lnqh4QqX4ZejGxyddsh', '4cicRwpo8XR3MsDkBp6QE3', '1mYuVwCn7kANLtTCDeZcfp', '30fV4xMJzFHts551WFdWBN', '3jAJa6HuL5U8AVYBXwu94F', '0pZ6LefsOYr0sVE05PkrMh', '1OJISKZHJBQrbEvkpQf5R0', '1uZoMjEdvVFarkSXmkYBn5', '7Elebnk4eNbbxraFGmxE9R', '7cC3X7fco9TZBxzOEbSZ9O', '2RypFDfxzPtuPLCxZ0Qa9H', '3uX6mHkBYcMvAqGV0X1YPo', '2rquYisEqft0wMukmgL8RM', '2fpkOZXgi9puMrUHFyqjYo', '7nimW24z8XG0I4ckiadcdG', '2VeSlHttFnxyQn6SQQRKI9', '6yugHBhN1AzOdTDabZyjY0', '26ZOmiveokhgN04gyF6pDP', '0jJXgnSA7Mc1HSAJBW9i5e', '5ew1meycfVv9y620N8v6HU', '3zRYkXTZ1RM8XclyCAoMMY', '6B9SEMrM58lLd7OypSzgBS', '4P3y9BKOVxDotWAJ3Tvvxk', '6sdH2KCFsEbnShojn0weC2', '2fxibHOEKMfUJumZcoK95l', '2PzNFF5P9ezfuGpuyHPdyV', '1kt1Xf4Q6q

In [55]:
def get_new_playlists():
    
    # store_user_playlists_info() # pause while updating the function
    user_playlist_info = get_user_playlists_info()
    new_df = pd.DataFrame(user_playlist_info)
    new_df['date'] = datetime.datetime.now()
    # read the csv file
    old_df = pd.read_csv('user_playlists_info.csv')

    """Iterate though the new_df and search for matching value from the old dataframe save the index where the first value matches. the value should continue to match."""
    # iterate through the new_df
    user_playlist_info = get_user_playlists_info()
    new_df = pd.DataFrame(user_playlist_info)
    new_df['date'] = datetime.datetime.now()
    # read the csv file
    old_df = pd.read_csv('user_playlists_info.csv')
    the_index = 0
    previous_index = 0
    for i in range(len(new_df)):
        song_id = new_df['playlist_id'][i]
        try:
            # get the index of the matching value
            index = old_df[old_df['playlist_id'] == song_id].index[0]
            # if the index is not the previous index + 1 and it is not the first index
            if i != previous_index+1 and i != 0:
                # save the index
                the_index = i
            previous_index = i
        except IndexError:
            pass
    changed = new_df[:the_index]
    # replace the rows before the index with the new rows, changed
    old_df[:the_index] = changed
    # save the new dataframe to the csv file
    # old_df.to_csv('user_playlists_info.csv', index=False)
    # return all the playlits that are new
    return changed

the_ones = get_new_playlists()


,playlist_name,playlist_id,playlist_description,playlist_tracks,playlist_owner,playlist_owner_id,playlist_public,playlist_collaborative,playlist_images,date


In [7]:

def find_function(name_of_the_playlist):
    functions = []
    multiFunctions = []
    for i in re.findall(r'\w+-\w+', name_of_the_playlist):
        multiFunctions.append(i)
    print(multiFunctions)

find_function("artist-swans -all  genre-hip-hop")



['artist-swans', 'genre-hip']


In [31]:
words = 'hey artist-swans -all'
# use re to get the words that has a dash between them
words = 
print(words)

['artist-swans']


In [20]:
""" Uesr can add commands in the playlist name or in the playlist discription, this function
searches for the all the possible commands and returns a list of the commands """
def get_commands(dataframe):
    # get the playlist name and description
    playlist_name = dataframe['playlist_name']
    playlist_description = dataframe['playlist_description']
    # List of commands to arrange the playlists
    rearrange = {   'most played' : ["top", "most played", "top played"],
                    'least played' : ["least played"],
                    'most recently played' : ["most recently played", "most recent", "recently played", "recent"],
                    'recently played' : ["recently played", "recent", "last played"],
                 }
    # command meaning
    key_meaning = { 'artist' : ['artist', 'a'],
            'album' : ['album', 'al'],
            'song' : ['song', 's'],
            'playlist' : ['playlist', 'p'],
            'genre' : ['genre', 'g'],
            'find' : ['find', 'f', 'search', 's'],
            'generate' : ['generate', 'g'],
    }
    # create a list of the commands that are in the playlist name
    name_commands = []
    # create a list of the commands that are in the playlist description
    description_commands = []

# arrage the songs that were played 
# def arrange_top_played(playlist_id, days=50):


,playlist_name,playlist_id,playlist_description,playlist_tracks,playlist_owner,playlist_owner_id,playlist_public,playlist_collaborative,playlist_images,date
0,Human Sadness Radio,37i9dQZF1E8JhGUxJHmvnq,,50,Spotify,spotify,False,False,https://seeded-session-images.scdn.co/v1/img/t...,2023-03-26 16:10:34.986707
1,Poorvi + Saket,37i9dQZF1EJLsvVumBJwJe,A blend of music for Saket and Poorvi. Updates...,50,Spotify,spotify,False,False,https://blend-playlist-covers.spotifycdn.com/v...,2023-03-26 16:10:34.986707
2,Best of Radiohead,7sb69mB0VFyz4t1PwkFGMC,,13,Saket,rt47etgc6xpwhhhb8575rth83,True,False,https://mosaic.scdn.co/640/ab67616d0000b2736c7...,2023-03-26 16:10:34.986707
3,You and me 🌈🌼🌸,6wM3aS0HC00gw7R8XhzbO5,,308,Poorvi,l57o12u24498uhn083lff5vy3,False,False,https://mosaic.scdn.co/640/ab67616d0000b27350d...,2023-03-26 16:10:34.986707


In [29]:
import requests
import json

# Set the target word and the context
word = "apple"
context = "general"

# Set the Owl API endpoint
url = "https://owlapi.herokuapp.com/api/v1/similar"

# Set the parameters for the request
params = {
    "word": word,
    "context": context
}

# Send a GET request to the Owl API
response = requests.get(url, params=params)

# Parse the JSON response
data = response.json()

# Print the most similar words
print("The most similar words to", word, "in", context, "context are:")
for item in data["similar_words"]:
    print(item["word"], "-", item["similarity"])

In [69]:

# # a functions that finds what difference were made in the new dataframe and the old one
# def find_difference(new_df, old_df):
    


In [ ]:


# returns a list of dictionaries containing the user's playlists information
    # def get_user_playlists_info():
    #     user_playlists = get_user_playlists()
    #     user_playlists_info = []
    #     for i in range(len(user_playlists['items'])):
    #         playlist = user_playlists['items'][i]
    #         playlist_info = {
    #             'playlist_name': playlist['name'],
    #             'playlist_id': playlist['id'],
    #             'playlist_description': playlist['description'],
    #             'playlist_tracks': playlist['tracks']['total'],
    #             'playlist_owner': playlist['owner']['display_name'],
    #             'playlist_owner_id': playlist['owner']['id'],
    #             'playlist_public': playlist['public'],
    #             'playlist_collaborative': playlist['collaborative'],
    #             'playlist_images': playlist['images'][0]['url']
    #         }
    #         user_playlists_info.append(playlist_info)
    #     return user_playlists_info

# stores the user's playlists in a csv file


In [46]:
tracks_info = get_recommendations_info(["79S80ZWgVhIPMCHuvl6SkA"], ["post-rock"], ["05JqOBN6XW4eFUVQlgR0I3"])


In [ ]:

# playlist-read-collaborative: Include collaborative playlists when requesting a user’s playlists1.
# playlist-modify-public: Write access to a user’s public playlists1.
# playlist-modify-private: Write access to a user’s private playlists1.


In [13]:
# dictionaries containing the user's playlists information
library_tracks_info = get_library_info(limit=50)

In [18]:
# get all the user's playlists
user_playlists = get_user_playlists_info()

href
items
limit
next
offset
previous
total


In [49]:
# iterate through the first track from tracks_info and print all the keys
for key in tracks_info[0]:
    print(key)
    


album
artists
available_markets
disc_number
duration_ms
explicit
external_ids
external_urls
href
id
is_local
name
popularity
preview_url
track_number
type
uri


In [4]:

# store the recently played tracks
store_recently_played_tracks()




In [20]:
# get the uers's playlists
user_playlists = get_user_playlists(playlist_read_private)

NameError: name 'playlist_read_private' is not defined

In [28]:
for i in user_playlists:
    print(i)

href
items
limit
next
offset
previous
total


In [13]:
# function read the most recently made playlist
def read_most_recent_playlist():
    playlists_list = get_user_playlists_info()
    # arrange the playlists by date
    playlists_list.sort(key=lambda x: x['playlist_id'], reverse=True)




In [102]:
# location of the folder to store the data



# # get the top 50 tracks played by the user
# def get_top_tracks(token):
#     top_tracks = token.current_user_top_tracks(limit=50)
#     return top_tracks


In [4]:
data_folder = r"C:\Users\saket\Documents\1.MY_DATA\spotify\spotify api data"
recently_played_file_name = 'recently_played_tracks.csv'

# function to add the recently played tracks in a csv file
def store_recently_played_tracks():
    recently_played_tracks = get_recently_played_tracks_info(recently_played)
    # create a dataframe from the list of recently played tracks
    recently_played_file_path = os.path.join(data_folder, recently_played_file_name)
    old_recently_played_tracks = pd.read_csv(recently_played_file_path)
    new_recently_played_tracks = pd.DataFrame(recently_played_tracks)
    # get the last played track from the old_recently_played_tracks
    last_played_at = old_recently_played_tracks['played_at'].iloc[-1]
    # reverse the new_recently_played_tracks
    new_recently_played_tracks = new_recently_played_tracks.iloc[::-1]

    # iterate though all the new_recently_played_tracks and find the one which is closest to the last_played_at
    for i in range(len(new_recently_played_tracks)):
        # get the difference between the last_played_at and the current track's played_at in hours
        diff = (timestamp_to_time(last_played_at) - timestamp_to_time(new_recently_played_tracks['played_at'].iloc[i])).total_seconds() / 3600
        # print(diff, new_recently_played_tracks['track_name'].iloc[i])
        # if the difference is negative, then the current track is the one which was last played
        if diff < 0:
            # get the index of the new_recently_played_tracks
            index = i
            # start the new_recently_played_tracks from the index
            new_recently_played_tracks = new_recently_played_tracks.iloc[index:]
            # print(new_recently_played_tracks['track_name'].iloc[index])
                # add the new_recently_played_tracks to the old_recently_played_tracks, use .concat() to avoid the index being repeated
            old_recently_played_tracks = pd.concat([old_recently_played_tracks, new_recently_played_tracks])
            # save the old_recently_played_tracks to the csv file
            old_recently_played_tracks.to_csv(recently_played_file_path, index=False)
            break

store_recently_played_tracks()



NameError: name 'recently_played' is not defined

In [ ]:
# import datetime
import datetime

# get all the useful information from the recently played tracks
def get_recently_played_tracks_info(recently_played):
    recently_played_tracks = []
    recently_played_token = get_spotify_token("user-read-recently-played")
    recently_played = get_recently_played_tracks(recently_played_token)
    for i in range(len(recently_played['items'])):
        track = recently_played['items'][i]['track']
        track_info = {
            'track_name': track['name'],
            'track_id': track['id'],
            'track_duration': track['duration_ms'],
            'track_popularity': track['popularity'],
            'track_explicit': track['explicit'],
            'album_name': track['album']['name'],
            'album_id': track['album']['id'],
            'album_release_date': track['album']['release_date'],
            'album_release_date_precision': track['album']['release_date_precision'],
            'album_total_tracks': track['album']['total_tracks'],
            'album_type': track['album']['album_type'],
            'artist_name': track['artists'][0]['name'],
            'artist_id': track['artists'][0]['id'],
            'played_at': recently_played['items'][i]['played_at']
        }
        # song name
        song_name = track['name']
        print(recently_played['items'][i]['played_at'], song_name)



        recently_played_tracks.append(track_info)
    return recently_played_tracks

# def store_recently_played_tracks():
    recently_played_tracks = get_recently_played_tracks_info(recently_played)
    # create a dataframe from the list of recently played tracks
    # recently_played_file_path = os.path.join(data_folder, recently_played_file_name)
    old_recently_played_tracks = pd.read_csv(recently_played_file_name)
    new_recently_played_tracks = pd.DataFrame(recently_played_tracks)
    # get the last played track from the old_recently_played_tracks
    last_played_at = old_recently_played_tracks['played_at'].iloc[-1]
    # reverse the new_recently_played_tracks
    new_recently_played_tracks = new_recently_played_tracks.iloc[::-1]

    # iterate though all the new_recently_played_tracks and find the one which is closest to the last_played_at
    for i in range(len(new_recently_played_tracks)):
        # get the difference between the last_played_at and the current track's played_at in hours
        diff = (timestamp_to_time(last_played_at) - timestamp_to_time(new_recently_played_tracks['played_at'].iloc[i])).total_seconds() / 3600
        # print(diff, new_recently_played_tracks['track_name'].iloc[i])
        # if the difference is negative, then the current track is the one which was last played
        if diff < 0:
            # get the index of the new_recently_played_tracks
            index = i
            # start the new_recently_played_tracks from the index
            new_recently_played_tracks = new_recently_played_tracks.iloc[index:]
            # print(new_recently_played_tracks['track_name'].iloc[index])
                # add the new_recently_played_tracks to the old_recently_played_tracks, use .concat() to avoid the index being repeated
            old_recently_played_tracks = pd.concat([old_recently_played_tracks, new_recently_played_tracks])
            # save the old_recently_played_tracks to the csv file
            old_recently_played_tracks.to_csv(recently_played_file_path, index=False)
            break


    # # iterate through all the library tracks
    # for i in range(len(library['items'])):
    #     # get the track
    #     track = library['items'][i]['track']
    #     # get the track info
    #     track_info = {
    #         'track_name': track['name'],
    #         'track_id': track['id'],
    #         'track_duration': track['duration_ms'],
    #         'track_popularity': track['popularity'],
    #         'track_explicit': track['explicit'],
    #         'album_name': track['album']['name'],
    #         'album_id': track['album']['id'],
    #         'album_release_date': track['album']['release_date'],
    #         'album_release_date_precision': track['album']['release_date_precision'],
    #         'album_total_tracks': track['album']['total_tracks'],
    #         'album_type': track['album']['album_type'],
    #         'artist_name': track['artists'][0]['name'],
    #         'artist_id': track['artists'][0]['id']
    #     }
    #     # add the track info to the library info
    #     library_info.append(track_info)


5


In [ ]:
time.time()

1678126686.844148

In [ ]:
# iterate through the first history item and print the track name
for item in history['items']:
    track = item['track']
    print(track['name'])

Oxygen
Hunter
Toxicity
Mother of the World
Screen Shot
Lunacy
The Blacker The Berry
Messenger
Simulation Swarm
Hey Jane
Song Against Sex
She Loves Us
Hacker
She Loves Us
Nosferatu Man
Oxygen
Mladic
Mladic
Mladic
IGOR'S THEME
Flume
Punkero Sonidero
money machine
1539 N. Calvert
Helplessness Blues
Mladic
The Black Hawk War, Or, How to Demolish an Entire Civilization and Still Feel Good About Yourself In the Morning, Or, We Apologize for the Inconvenience But You're Going to Have to Leave Now, Or...
Concerning the UFO sighting near Highland, Illinois
King of Carrot Flowers Pts. 2 & 3
A short reprise for Mary Todd, who went insane, but for very good reasons
Jacksonville
John Wayne Gacy, Jr.
Come On! Feel the Illinoise! Part I: The World's Columbian Exposition Part II: Carl Sandburg Visits Me In A Dream
The Black Hawk War, Or, How to Demolish an Entire Civilization and Still Feel Good About Yourself In the Morning, Or, We Apologize for the Inconvenience But You're Going to Have to Leave Now